In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/BenchmarkingSample20230607.csv')
df

,Unit,Period,Days,Account,Amount
0,00040302CD88,1,30,Sales,89952
1,00040302CD88,2,31,Sales,90594
2,00040302CD88,3,30,Sales,81909
3,00040302CD88,4,31,Sales,87418
4,00040302CD88,5,31,Sales,91109
...,...,...,...,...,...
6043,0004030BEB57,20,30,Other Non-controllables,1455
6044,0004030BEB57,21,31,Other Non-controllables,7456
6045,0004030BEB57,22,31,Other Non-controllables,2037
6046,0004030BEB57,23,28,Other Non-controllables,2037


In [3]:
# reshape df
unmelt = df.pivot_table(index=['Unit', 'Period', 'Days'], columns='Account', values='Amount', aggfunc='sum')
unmelt = unmelt.reset_index()
unmelt

Account,Unit,Period,Days,Advertising,Cost of Sales - Food,Cost of Sales - Paper,Crew Labor,Maintenance & Repair,Management Labor,Other Controllables,Other Labor-related Costs,Other Non-controllables,Outside Services,Promotion,Rent,Sales,Utilities
0,00040302CD88,1,30,3778,20557,2190,486,5135,3983,3167,3429,7879,8462,401,-610,89952,5690
1,00040302CD88,2,31,3805,20876,2058,1376,1553,4255,3899,3699,7918,5329,445,-178,90594,4917
2,00040302CD88,3,30,3440,19531,2291,178,2321,4241,3646,3614,7471,5062,553,-527,81909,4692
3,00040302CD88,4,31,3672,20999,2056,36597,3412,5515,2581,6944,7537,5967,874,-353,87418,5699
4,00040302CD88,5,31,3827,21357,2312,29942,3345,5484,3772,7109,6385,5727,635,-167,91109,5109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0004030BEB57,20,30,15729,99236,10557,75243,8789,3997,2729,15678,26672,11476,-3,64375,374497,6230
212,0004030BEB57,21,31,12227,108507,13137,94509,1704,4989,6187,15246,39347,16172,-1144,81337,424685,6480
213,0004030BEB57,22,31,15255,101713,10812,82894,3256,4278,4408,16168,28140,11189,2665,58308,363205,8236
214,0004030BEB57,23,28,15020,98547,11558,83338,3605,5253,3443,16454,27999,10646,608,56786,357611,2656


In [4]:
columns_to_average = unmelt.columns.drop(['Unit', 'Period', 'Days', 'Sales'])
for col in columns_to_average:
    unmelt[f'Average daily {col}'] = (unmelt[col] / unmelt['Days']).round(2)
unmelt.to_excel('data/output.xlsx', sheet_name='unmelt')
old_columns = unmelt.columns
old_columns

Index(['Unit', 'Period', 'Days', 'Advertising', 'Cost of Sales - Food',
       'Cost of Sales - Paper', 'Crew Labor', 'Maintenance & Repair',
       'Management Labor', 'Other Controllables', 'Other Labor-related Costs',
       'Other Non-controllables', 'Outside Services', 'Promotion', 'Rent',
       'Sales', 'Utilities', 'Average daily Advertising',
       'Average daily Cost of Sales - Food',
       'Average daily Cost of Sales - Paper', 'Average daily Crew Labor',
       'Average daily Maintenance & Repair', 'Average daily Management Labor',
       'Average daily Other Controllables',
       'Average daily Other Labor-related Costs',
       'Average daily Other Non-controllables',
       'Average daily Outside Services', 'Average daily Promotion',
       'Average daily Rent', 'Average daily Utilities'],
      dtype='object', name='Account')

In [5]:
# Add columns to the dataframe as per the Example P & L excel file
unmelt['Total Cost of Sales'] = unmelt.iloc[:, 3:5].sum(axis=1)
unmelt['Gross Profit'] = unmelt['Sales'] - unmelt['Total Cost of Sales']
unmelt['Total Labor'] = unmelt.iloc[:, [5, 7, 9]].sum(axis=1)
unmelt['Total Prime Costs'] = unmelt['Total Cost of Sales'] + unmelt['Total Labor']
unmelt['Total Controllable Costs'] = unmelt.iloc[:, [2, 5, 7, 9, 10, 11, 12, 14]].sum(axis = 1)
unmelt['Controllable Profit'] = unmelt['Gross Profit'] - unmelt['Total Controllable Costs']
unmelt['Total Non-controllable Costs'] = unmelt['Cost of Sales - Food'] + unmelt['Cost of Sales - Paper']
unmelt['Store EBITDA'] = unmelt['Gross Profit'] - unmelt['Total Controllable Costs'] - unmelt['Total Non-controllable Costs']
new_columns = unmelt.columns.drop(old_columns)
new_columns

Index(['Total Cost of Sales', 'Gross Profit', 'Total Labor',
       'Total Prime Costs', 'Total Controllable Costs', 'Controllable Profit',
       'Total Non-controllable Costs', 'Store EBITDA'],
      dtype='object', name='Account')

In [6]:
unmelt.sort_values(by=['Unit', 'Total Cost of Sales'])
unmelt[pd.Index(['Unit']).append(new_columns)]

Account,Unit,Total Cost of Sales,Gross Profit,Total Labor,Total Prime Costs,Total Controllable Costs,Controllable Profit,Total Non-controllable Costs,Store EBITDA
0,00040302CD88,24335,65617,10492,34827,29682,35935,22747,13188
1,00040302CD88,24681,65913,7510,32191,24309,41604,22934,18670
2,00040302CD88,22971,58938,8258,31229,23908,35030,21822,13208
3,00040302CD88,24671,62747,8049,32720,28175,34572,23055,11517
4,00040302CD88,25184,65925,9429,34613,28514,37411,23669,13742
...,...,...,...,...,...,...,...,...,...
211,0004030BEB57,114965,259532,22075,137040,140306,119226,109793,9433
212,0004030BEB57,120734,303951,21028,141762,173161,130790,121644,9146
213,0004030BEB57,116968,246237,18476,135444,132312,113925,112525,1400
214,0004030BEB57,113567,244044,18606,132173,130519,113525,110105,3420


In [7]:
# Group data by unit
grouped_data = unmelt.drop(columns=['Period', 'Days']).groupby('Unit').sum()
grouped_data

Account,Advertising,Cost of Sales - Food,Cost of Sales - Paper,Crew Labor,Maintenance & Repair,Management Labor,Other Controllables,Other Labor-related Costs,Other Non-controllables,Outside Services,...,Average daily Rent,Average daily Utilities,Total Cost of Sales,Gross Profit,Total Labor,Total Prime Costs,Total Controllable Costs,Controllable Profit,Total Non-controllable Costs,Store EBITDA
Unit,,,,,,,,,,,,,,,,,,,,,
00040302CD88,109741,720626,82141,701391,78634,105206,71672,154874,225612,178464,...,-1946.42,4320.97,830367,1841416,232447,1062814,732839,1108577,802767,305810
00040302CE36,229150,1347519,148997,1284266,77781,126972,83111,259812,419873,192744,...,20680.12,4650.89,1576669,3967273,309889,1886558,1812090,2155183,1496516,658667
00040302CF03,149145,882289,103966,906952,78844,110631,81785,189284,316942,164199,...,1375.30,4586.04,1031434,2600347,264595,1296029,977797,1622550,986255,636295
00040302CF28,181308,1120990,136565,1055731,91160,139168,83189,225791,348075,255092,...,8980.12,4684.85,1302298,3101184,310914,1613212,1413578,1687606,1257555,430051
00040302D006,63925,388096,46804,280791,56467,78184,49276,79529,202873,82064,...,-2563.60,2167.13,452021,1116838,152547,604568,439794,677044,434900,242144
00040302D014,252284,1422862,148973,1314039,82393,149005,77871,263992,459970,92344,...,30939.68,5213.66,1675146,4428051,309237,1984383,2067499,2360552,1571835,788717
0004030A635E,510987,2904984,313677,2494044,117275,197547,137337,542492,848481,193584,...,71272.49,6484.50,3415971,8922744,568289,3984260,4322363,4600381,3218661,1381720
0004030B92EE,321873,1901998,204654,1737196,124223,147557,106694,341753,551801,214347,...,42008.25,5467.47,2223871,5552155,435571,2659442,2821942,2730213,2106652,623561
0004030BEB57,391903,2266305,252347,2086903,120814,135831,110448,407477,699612,299330,...,52207.40,6037.78,2658208,6806344,483609,3141817,3479587,3326757,2518652,808105


In [8]:
columns_to_convert = grouped_data.columns.drop('Sales')
for col in columns_to_convert:
    grouped_data[f'{col}(%)'] = (grouped_data[col] * 100 / grouped_data['Sales']).round(2)
df_existing = pd.read_excel('data/output.xlsx')
with pd.ExcelWriter('data/output.xlsx', mode='a') as writer:
    grouped_data.to_excel(writer, sheet_name='grouped_data')